# 🚀 Google Colab Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/10_algorithmes_genetiques/10_demo_ag_base.ipynb)

**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    print('📦 Installation des packages...')        # Packages ML de base    !pip install -q numpy pandas matplotlib seaborn scikit-learn        # Détection du chapitre et installation des dépendances spécifiques    notebook_name = '10_demo_ag_base.ipynb'  # Sera remplacé automatiquement        # Ch 06-08 : Deep Learning    if any(x in notebook_name for x in ['06_', '07_', '08_']):        !pip install -q torch torchvision torchaudio        # Ch 08 : NLP    if '08_' in notebook_name:        !pip install -q transformers datasets tokenizers        if 'rag' in notebook_name:            !pip install -q sentence-transformers faiss-cpu rank-bm25        # Ch 09 : Reinforcement Learning    if '09_' in notebook_name:        !pip install -q gymnasium[classic-control]        # Ch 04 : Boosting    if '04_' in notebook_name and 'boosting' in notebook_name:        !pip install -q xgboost lightgbm catboost        # Ch 05 : Clustering avancé    if '05_' in notebook_name:        !pip install -q umap-learn        # Ch 11 : Séries temporelles    if '11_' in notebook_name:        !pip install -q statsmodels prophet        # Ch 12 : Vision avancée    if '12_' in notebook_name:        !pip install -q ultralytics timm segmentation-models-pytorch        # Ch 13 : Recommandation    if '13_' in notebook_name:        !pip install -q scikit-surprise implicit        # Ch 14 : MLOps    if '14_' in notebook_name:        !pip install -q mlflow fastapi pydantic        print('✅ Installation terminée !')else:    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 10 - Algorithmes Génétiques (Base)

**Objectifs :**
- Comprendre les principes des algorithmes génétiques (AG)
- Implémenter un AG from scratch
- Appliquer les AG à l'optimisation de fonctions
- Résoudre le problème du voyageur de commerce (TSP)

**Concepts clés :** Population, Fitness, Sélection, Crossover, Mutation

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple, Callable
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Imports réussis")

## 1. Algorithme Génétique - Implémentation

### 1.1 Classe GeneticAlgorithm

In [ ]:
class GeneticAlgorithm:
    """Algorithme Génétique simple pour optimisation."""
    
    def __init__(self, 
                 fitness_func: Callable,
                 pop_size: int = 100,
                 chrom_length: int = 10,
                 crossover_rate: float = 0.8,
                 mutation_rate: float = 0.01,
                 elitism: int = 2,
                 maximize: bool = True):
        """
        Args:
            fitness_func: Fonction de fitness f(chromosome) -> float
            pop_size: Taille de la population
            chrom_length: Longueur du chromosome (nombre de gènes)
            crossover_rate: Probabilité de crossover
            mutation_rate: Probabilité de mutation par gène
            elitism: Nombre de meilleurs individus préservés
            maximize: True pour maximiser, False pour minimiser
        """
        self.fitness_func = fitness_func
        self.pop_size = pop_size
        self.chrom_length = chrom_length
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.elitism = elitism
        self.maximize = maximize
        
        # Historique
        self.history = {'best_fitness': [], 'avg_fitness': []}
        
    def initialize_population(self) -> np.ndarray:
        """Créer population initiale aléatoire (binaire)."""
        return np.random.randint(0, 2, size=(self.pop_size, self.chrom_length))
    
    def evaluate_population(self, population: np.ndarray) -> np.ndarray:
        """Évaluer la fitness de chaque individu."""
        fitness = np.array([self.fitness_func(ind) for ind in population])
        return fitness if self.maximize else -fitness
    
    def selection_tournament(self, population: np.ndarray, 
                           fitness: np.ndarray, tournament_size: int = 3) -> np.ndarray:
        """Sélection par tournoi."""
        idx = np.random.choice(len(population), tournament_size, replace=False)
        winner_idx = idx[np.argmax(fitness[idx])]
        return population[winner_idx]
    
    def crossover_one_point(self, parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Crossover à un point."""
        if np.random.rand() < self.crossover_rate:
            point = np.random.randint(1, self.chrom_length)
            child1 = np.concatenate([parent1[:point], parent2[point:]])
            child2 = np.concatenate([parent2[:point], parent1[point:]])
            return child1, child2
        return parent1.copy(), parent2.copy()
    
    def mutate(self, chromosome: np.ndarray) -> np.ndarray:
        """Mutation bit-flip."""
        mask = np.random.rand(self.chrom_length) < self.mutation_rate
        chromosome[mask] = 1 - chromosome[mask]
        return chromosome
    
    def evolve(self, generations: int, verbose: bool = True) -> Tuple[np.ndarray, float]:
        """Boucle d'évolution principale."""
        # Initialisation
        population = self.initialize_population()
        
        for gen in range(generations):
            # Évaluation
            fitness = self.evaluate_population(population)
            
            # Stats
            best_idx = np.argmax(fitness)
            best_fitness = fitness[best_idx]
            avg_fitness = fitness.mean()
            
            self.history['best_fitness'].append(best_fitness)
            self.history['avg_fitness'].append(avg_fitness)
            
            if verbose and gen % 10 == 0:
                print(f"Gen {gen:3d}: Best={best_fitness:.4f}, Avg={avg_fitness:.4f}")
            
            # Élitisme
            elite_idx = np.argsort(fitness)[-self.elitism:]
            elites = population[elite_idx]
            
            # Nouvelle génération
            new_population = []
            
            while len(new_population) < self.pop_size - self.elitism:
                # Sélection
                parent1 = self.selection_tournament(population, fitness)
                parent2 = self.selection_tournament(population, fitness)
                
                # Crossover
                child1, child2 = self.crossover_one_point(parent1, parent2)
                
                # Mutation
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                
                new_population.extend([child1, child2])
            
            # Combiner élites + nouveaux
            population = np.vstack([elites, new_population[:self.pop_size - self.elitism]])
        
        # Retourner meilleur individu
        final_fitness = self.evaluate_population(population)
        best_idx = np.argmax(final_fitness)
        best_individual = population[best_idx]
        best_score = final_fitness[best_idx] if self.maximize else -final_fitness[best_idx]
        
        return best_individual, best_score
    
    def plot_history(self):
        """Visualiser l'évolution."""
        plt.figure(figsize=(10, 6))
        plt.plot(self.history['best_fitness'], label='Best Fitness', linewidth=2)
        plt.plot(self.history['avg_fitness'], label='Avg Fitness', linewidth=2, alpha=0.7)
        plt.xlabel('Generation')
        plt.ylabel('Fitness')
        plt.title('Évolution de la Fitness')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

print("✅ Classe GeneticAlgorithm implémentée")

## 2. Exemple 1 : Maximiser la Somme des Bits

**Problème :** Maximiser $f(x) = \sum_{i=1}^n x_i$ (fonction triviale pour tester)

In [ ]:
# Fonction de fitness simple: compte le nombre de 1s
def fitness_ones(chromosome):
    return chromosome.sum()

# Créer et exécuter AG
ga_ones = GeneticAlgorithm(
    fitness_func=fitness_ones,
    pop_size=50,
    chrom_length=20,
    crossover_rate=0.8,
    mutation_rate=0.05,
    maximize=True
)

best, score = ga_ones.evolve(generations=50, verbose=True)

print(f"\n=== Résultat ===")
print(f"Meilleur individu : {best}")
print(f"Score : {score:.0f}/{len(best)} bits à 1")

ga_ones.plot_history()

## 3. Exemple 2 : Optimisation de Fonction Continue

**Problème :** Maximiser $f(x) = x \sin(10\pi x) + 1$ sur $[0, 2]$

**Encodage :** Binaire → Conversion en float

In [ ]:
def binary_to_float(chromosome, x_min=0, x_max=2):
    """Convertir binaire en float."""
    decimal = int(''.join(map(str, chromosome)), 2)
    max_val = 2**len(chromosome) - 1
    return x_min + (x_max - x_min) * decimal / max_val

def fitness_continuous(chromosome):
    """Fitness pour optimisation continue."""
    x = binary_to_float(chromosome)
    return x * np.sin(10 * np.pi * x) + 1

# Visualiser la fonction
x_range = np.linspace(0, 2, 1000)
y_range = x_range * np.sin(10 * np.pi * x_range) + 1

plt.figure(figsize=(10, 6))
plt.plot(x_range, y_range, linewidth=2)
plt.xlabel('x')
plt.ylabel('f(x) = x sin(10πx) + 1')
plt.title('Fonction à Optimiser')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Maximum théorique : x ≈ {x_range[np.argmax(y_range)]:.4f}, f(x) ≈ {y_range.max():.4f}")

In [ ]:
# Exécuter AG
ga_cont = GeneticAlgorithm(
    fitness_func=fitness_continuous,
    pop_size=100,
    chrom_length=20,  # Précision: 2^20 ≈ 1 million de points
    crossover_rate=0.9,
    mutation_rate=0.01,
    maximize=True
)

best_cont, score_cont = ga_cont.evolve(generations=100, verbose=True)

x_best = binary_to_float(best_cont)
print(f"\n=== Résultat ===")
print(f"Meilleur x : {x_best:.6f}")
print(f"f(x) : {score_cont:.6f}")
print(f"Écart avec théorique : {abs(score_cont - y_range.max()):.6f}")

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Solution trouvée
axes[0].plot(x_range, y_range, linewidth=2, label='f(x)')
axes[0].scatter(x_best, score_cont, color='red', s=200, zorder=5, label='Solution AG')
axes[0].set_xlabel('x')
axes[0].set_ylabel('f(x)')
axes[0].set_title('Solution Trouvée par AG')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Convergence
axes[1].plot(ga_cont.history['best_fitness'], linewidth=2, label='Best')
axes[1].plot(ga_cont.history['avg_fitness'], linewidth=2, alpha=0.7, label='Avg')
axes[1].set_xlabel('Generation')
axes[1].set_ylabel('Fitness')
axes[1].set_title('Convergence')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Problème du Voyageur de Commerce (TSP)

**Objectif :** Trouver le plus court chemin visitant toutes les villes une seule fois

### 4.1 Génération de Villes

In [ ]:
class TSPGeneticAlgorithm:
    """AG adapté pour TSP (permutations)."""
    
    def __init__(self, cities: np.ndarray, pop_size: int = 100, 
                 crossover_rate: float = 0.8, mutation_rate: float = 0.2):
        self.cities = cities
        self.n_cities = len(cities)
        self.pop_size = pop_size
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.history = {'best_distance': [], 'avg_distance': []}
        
        # Matrice de distances
        self.dist_matrix = self._compute_distance_matrix()
    
    def _compute_distance_matrix(self):
        """Calculer matrice des distances euclidiennes."""
        n = self.n_cities
        dist = np.zeros((n, n))
        for i in range(n):
            for j in range(i+1, n):
                d = np.linalg.norm(self.cities[i] - self.cities[j])
                dist[i, j] = dist[j, i] = d
        return dist
    
    def fitness(self, tour: np.ndarray) -> float:
        """Calculer distance totale du tour (à minimiser)."""
        distance = sum(self.dist_matrix[tour[i], tour[i+1]] 
                      for i in range(self.n_cities - 1))
        distance += self.dist_matrix[tour[-1], tour[0]]  # Retour au début
        return -distance  # Négatif car on maximise
    
    def initialize_population(self):
        """Créer population de tours aléatoires."""
        return np.array([np.random.permutation(self.n_cities) 
                        for _ in range(self.pop_size)])
    
    def selection_tournament(self, population, fitness, k=3):
        idx = np.random.choice(len(population), k, replace=False)
        winner = idx[np.argmax(fitness[idx])]
        return population[winner]
    
    def crossover_order(self, parent1, parent2):
        """Order Crossover (OX)."""
        if np.random.rand() > self.crossover_rate:
            return parent1.copy(), parent2.copy()
        
        size = len(parent1)
        start, end = sorted(np.random.choice(size, 2, replace=False))
        
        # Enfant 1
        child1 = np.full(size, -1)
        child1[start:end] = parent1[start:end]
        
        ptr = end
        for city in np.roll(parent2, -end):
            if city not in child1:
                child1[ptr % size] = city
                ptr += 1
        
        # Enfant 2 (symétrique)
        child2 = np.full(size, -1)
        child2[start:end] = parent2[start:end]
        
        ptr = end
        for city in np.roll(parent1, -end):
            if city not in child2:
                child2[ptr % size] = city
                ptr += 1
        
        return child1, child2
    
    def mutate_swap(self, tour):
        """Mutation par échange de 2 villes."""
        if np.random.rand() < self.mutation_rate:
            i, j = np.random.choice(len(tour), 2, replace=False)
            tour[i], tour[j] = tour[j], tour[i]
        return tour
    
    def evolve(self, generations=500, verbose=True):
        population = self.initialize_population()
        
        for gen in range(generations):
            fitness = np.array([self.fitness(ind) for ind in population])
            
            best_idx = np.argmax(fitness)
            best_distance = -fitness[best_idx]
            avg_distance = -fitness.mean()
            
            self.history['best_distance'].append(best_distance)
            self.history['avg_distance'].append(avg_distance)
            
            if verbose and gen % 50 == 0:
                print(f"Gen {gen:3d}: Best={best_distance:.2f}, Avg={avg_distance:.2f}")
            
            # Élitisme
            elites = population[np.argsort(fitness)[-2:]]
            
            new_population = []
            while len(new_population) < self.pop_size - 2:
                p1 = self.selection_tournament(population, fitness)
                p2 = self.selection_tournament(population, fitness)
                c1, c2 = self.crossover_order(p1, p2)
                c1 = self.mutate_swap(c1)
                c2 = self.mutate_swap(c2)
                new_population.extend([c1, c2])
            
            population = np.vstack([elites, new_population[:self.pop_size-2]])
        
        final_fitness = np.array([self.fitness(ind) for ind in population])
        best_idx = np.argmax(final_fitness)
        return population[best_idx], -final_fitness[best_idx]

print("✅ Classe TSPGeneticAlgorithm implémentée")

In [ ]:
# Générer villes aléatoires
np.random.seed(42)
n_cities = 20
cities = np.random.rand(n_cities, 2) * 100

# Visualiser
plt.figure(figsize=(8, 8))
plt.scatter(cities[:, 0], cities[:, 1], s=200, c='red', zorder=5)
for i, (x, y) in enumerate(cities):
    plt.text(x+1, y+1, str(i), fontsize=12)
plt.xlabel('X')
plt.ylabel('Y')
plt.title(f'TSP : {n_cities} Villes')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.show()

print(f"Villes générées : {n_cities}")

### 4.2 Résolution avec AG

In [ ]:
# Créer et exécuter AG pour TSP
tsp_ga = TSPGeneticAlgorithm(
    cities=cities,
    pop_size=200,
    crossover_rate=0.9,
    mutation_rate=0.1
)

best_tour, best_distance = tsp_ga.evolve(generations=500, verbose=True)

print(f"\n=== Meilleur Tour Trouvé ===")
print(f"Distance : {best_distance:.2f}")
print(f"Tour : {best_tour}")

In [ ]:
# Visualisation finale
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Plot 1: Meilleur tour
tour_coords = cities[best_tour]
tour_coords = np.vstack([tour_coords, tour_coords[0]])  # Boucle

axes[0].plot(tour_coords[:, 0], tour_coords[:, 1], 'b-', linewidth=2, alpha=0.6)
axes[0].scatter(cities[:, 0], cities[:, 1], s=200, c='red', zorder=5)
axes[0].scatter(cities[best_tour[0], 0], cities[best_tour[0], 1], 
               s=300, c='green', marker='*', zorder=6, label='Départ')
for i, (x, y) in enumerate(cities):
    axes[0].text(x+1, y+1, str(i), fontsize=10)
axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')
axes[0].set_title(f'Meilleur Tour (Distance={best_distance:.2f})')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].axis('equal')

# Plot 2: Convergence
axes[1].plot(tsp_ga.history['best_distance'], linewidth=2, label='Best')
axes[1].plot(tsp_ga.history['avg_distance'], linewidth=2, alpha=0.7, label='Avg')
axes[1].set_xlabel('Generation')
axes[1].set_ylabel('Distance Totale')
axes[1].set_title('Convergence TSP')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ TSP résolu avec succès !")

## 5. Récapitulatif

### Composants d'un AG :
1. **Population** : Ensemble d'individus (chromosomes)
2. **Fitness** : Fonction d'évaluation
3. **Sélection** : Tournoi, roulette, rang
4. **Crossover** : Un point, deux points, uniforme, order (TSP)
5. **Mutation** : Bit-flip, swap, inversion
6. **Élitisme** : Préserver les meilleurs

### Avantages :
- Pas besoin de gradient
- Robuste aux optimums locaux
- Applicable à problèmes discrets et continus

### Limites :
- Lent pour grandes dimensions
- Pas de garantie de convergence
- Nécessite tuning des hyperparamètres

### Prochaine étape :
Voir **10_demo_applications.ipynb** pour hyperparameter tuning et feature selection

In [ ]:
print("✅ Notebook terminé !")
print("\nVous maîtrisez maintenant :")
print("  - Algorithmes génétiques (théorie et implémentation)")
print("  - Optimisation de fonctions continues")
print("  - TSP avec Order Crossover")
print("  - Sélection, Crossover, Mutation")